[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/abhibambhaniya/GenZ-LLM-Analyzer/blob/main/notebook/GenZ_on_colab.ipynb)

In [ ]:
## Run only once
!git clone https://github.com/abhibambhaniya/GenZ-LLM-Analyzer.git
!pip install seaborn
!pip install plotnine
!pip install tqdm
!pip install plotly
!pip install paretoset

In [9]:
# Import necessary libraries

import os, sys, warnings
script_dir = os.getcwd()
module_path = script_dir
sys.path.insert(0, '/content/GenZ-LLM-Analyzer')

from src import decode_moddeling, prefill_moddeling
from Systems.system_configs import *
import pandas as pd
from plotnine import *
import plotnine as p9
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import plotly.express as px
from plotnine import *
import plotnine as p9


In [11]:
batch_size_list = [1, 2, 4, 8]
input_token_list = [100, 250, 500]
model_box = ['llama_7b', 'llama_70b']

data = []
for model in model_box:
  for batch_size in tqdm(batch_size_list):
    for input_token in input_token_list:
                if model == 'llama_7b':
                    tp, pp = 1, 1
                elif model == 'llama_70b':
                    tp, pp = 2, 1
                prefill_outputs = prefill_moddeling(model = model, batch_size = batch_size,
                                        input_tokens = input_token, output_tokens = 100,
                                        system_name = H100_GPU, system_eff = 1,
                                        bits='bf16',
                                        tensor_parallel = tp, pipeline_parallel = pp,  debug=False, time_breakdown=True)
                TTFT = prefill_outputs['Latency'] / 1000  ## Secs
                decode_outputs = decode_moddeling(model = model, batch_size = batch_size, Bb = 1 ,
                                        input_tokens = input_token, output_tokens = 100,
                                        system_name = H100_GPU, system_eff=1,
                                        bits='bf16',
                                        tensor_parallel = tp, pipeline_parallel = pp, debug=False, time_breakdown=True)
                TPOT = decode_outputs['Latency'] / 1000  ## Secs
                Latency = TTFT + TPOT*100
                data.append([model, tp, pp ,batch_size, input_token, TTFT, TPOT, Latency, decode_outputs['Throughput']] + prefill_outputs['Runtime_breakdown'] + decode_outputs['Runtime_breakdown'])

assert len(data) > 0, "No Model fits in the given # of GPUs. Increase GPUs or use different Model"
data_df = pd.DataFrame(data, columns = ['Model','TP', 'PP', 'Batch', 'Input Size', 'TTFT(sec)', 'TPOT(sec)', 'Latency(sec)', 'Decode Tokens/s', 'Prefill GEMM Time(ms)', 'Prefill Attn Time(ms)', 'Prefill Communication Time(ms)', 'Decode GEMM Time(ms)', 'Decode Attn Time(ms)', 'Decode Communication Time(ms)'])
data_df

100%|██████████| 4/4 [00:00<00:00,  6.14it/s]


,Model,TP,PP,Batch,Input Size,TTFT(sec),TPOT(sec),Latency(sec),Decode Tokens/s,Prefill GEMM Time(ms),Prefill Attn Time(ms),Prefill Communication Time(ms),Decode GEMM Time(ms),Decode Attn Time(ms),Decode Communication Time(ms)
0,llama_7b,1,1,1,100,0.003701,0.003571,0.360782,280.048635,3.670313,0.030842,0.000000,3.549019,0.029052,0.000000
1,llama_7b,1,1,1,250,0.003939,0.003592,0.363177,278.366837,3.854092,0.085052,0.000000,3.549019,0.050626,0.000000
2,llama_7b,1,1,1,500,0.006745,0.003628,0.369578,275.608284,6.548034,0.196594,0.000000,3.549019,0.086582,0.000000
3,llama_7b,1,1,2,100,0.003855,0.003594,0.363237,556.510504,3.792832,0.061683,0.000000,3.550244,0.058104,0.000000
4,llama_7b,1,1,2,250,0.006718,0.003637,0.370415,549.908337,6.548034,0.170103,0.000000,3.550244,0.101251,0.000000
5,llama_7b,1,1,2,500,0.013489,0.003709,0.384377,539.246087,13.096067,0.393188,0.000000,3.550244,0.173163,0.000000
6,llama_7b,1,1,4,100,0.005362,0.003640,0.369347,1098.946086,5.238427,0.123367,0.000000,3.552695,0.116208,0.000000
7,llama_7b,1,1,4,250,0.013436,0.003726,0.386051,1073.495407,13.096067,0.340206,0.000000,3.552695,0.202503,0.000000
8,llama_7b,1,1,4,500,0.026979,0.003870,0.413975,1033.599881,26.192135,0.786376,0.000000,3.552695,0.346327,0.000000
9,llama_7b,1,1,8,100,0.010724,0.003732,0.383914,2143.675646,10.476854,0.246734,0.000000,3.557596,0.232416,0.000000
